## CMPINF 2110 Spring 2021 - Homework 03

### SOLUTION GUIDE

Dr. Joseph P. Yurko

### Overview

This notebook provides the solutions for how to "decompose" or "break-up" the tidy data sets from homework 02 into smaller, unique, "constituent" tables. The tables created in this notebook are imported into MySQL in a separate script, `homework_03_sql_solutions.sql`. Please see that script for the MySQL related questions.

The corresponding data relationship diagrams are provided in the `homework_03_data_relationship_diagrams.pdf` file.

## Import Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

## Application 1 - TB data

### Tidy the data
We will first read in the original TB data from Homework 02 and then put "tidy" the data into the long-format.

In [2]:
tb = pd.read_csv('../hw02/tidy_data_table_9_use.csv' )

The steps and detailed discussion around the process of tidying the data set is provided in the solutions to homework 02. In these solutions we will simply get the data into the tidy format to serve as the starting point for homework 03.

In [3]:
feature_names = tb.drop(columns=['country', 'year']).columns.to_list()

lf = tb.melt(id_vars=['year', 'country'], value_vars=feature_names)

lf[['gender', 'age_group_word']] = lf.variable.str.split( '_', expand=True )

age_info = lf.groupby(['age_group_word']).size().reset_index(name='num_rows')

age_info.drop(columns=['num_rows'], inplace=True)

age_info['age_group'] = ['0-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65+', 'unknown']

lf = pd.merge( lf, age_info, on='age_group_word', how='left')

tidy_tb = lf.loc[:, ['country', 'year', 'gender', 'age_group', 'value']].copy()

tidy_tb.rename(columns={'gender': 'sex', 'age_group': 'age', 'value': 'cases'}, inplace=True)

The `.info()` method for the tidy long-format data set, `tidy_tb`, is given below.

In [4]:
tidy_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92304 entries, 0 to 92303
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  91936 non-null  object 
 1   year     92304 non-null  int64  
 2   sex      92304 non-null  object 
 3   age      92304 non-null  object 
 4   cases    34160 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ MB


The first few and last few rows are displayed with the `.head()` and `.tail()` methods below.

In [5]:
tidy_tb.head()

,country,year,sex,age,cases
0,AD,1989,f,0-14,NaN
1,AD,1990,f,0-14,NaN
2,AD,1991,f,0-14,NaN
3,AD,1992,f,0-14,NaN
4,AD,1993,f,0-14,NaN


In [6]:
tidy_tb.tail()

,country,year,sex,age,cases
92299,NaN,2004,m,unknown,NaN
92300,NaN,2005,m,unknown,NaN
92301,NaN,2006,m,unknown,NaN
92302,NaN,2007,m,unknown,NaN
92303,NaN,2008,m,unknown,0.0


### Decompose the tidy data

One row in our tidy data set gives the number of TB cases in a year within a country for a particular age group and gender. Let's decompose the tidy data set into several smaller data sets containing the unique information about the countries, years, genders, and age groups. This way we can have separate "information" tables storing information associated with each of the *entities*, allowing additional *attributes* of those entities to be stored separate from the TB cases.

Let's begin with counting the number of unique values for the columns in `tidy_tb`.

In [7]:
tidy_tb.nunique()

country     212
year         29
sex           2
age           8
cases      3400
dtype: int64

#### Age 

We already created the "age information" table when we went through and created the tidy data set. The `age_info` DataFrame is displayed below. Notice it that it has the same number of rows as there are unique values for `age` in `tidy_tb`.

In [8]:
age_info

,age_group_word,age_group
0,014,0-14
1,1524,15-24
2,2534,25-34
3,3544,35-44
4,4554,45-54
5,5564,55-64
6,65,65+
7,u,unknown


Let's add a column to the `age_info` table which serves as the unique identifer (ID) for each age group. The `age_info.index` provides that unique identifer, but let's move that value directly into the DataFrame. We will also set that unique ID to start at 1 rather than 0.

In [9]:
age_info['age_id'] = age_info.index + 1

In [10]:
age_info

,age_group_word,age_group,age_id
0,014,0-14,1
1,1524,15-24,2
2,2534,25-34,3
3,3544,35-44,4
4,4554,45-54,5
5,5564,55-64,6
6,65,65+,7
7,u,unknown,8


Let's reorganize the columns such that the `age_id` is listed first.

In [11]:
age_info = age_info.loc[:, ['age_id', 'age_group', 'age_group_word']].copy()

And next let's rename the `age_group_word` to be `original_tb_word` that way we know this was the name of the age group in the original TB data set. You did not need to do this, but it lets us store the original name before we gave it the easier to read age group name.

In [12]:
age_info.rename(columns={'age_group_word': 'original_tb_word'}, inplace=True)

The age information table is displayed below.

In [13]:
age_info

,age_id,age_group,original_tb_word
0,1,0-14,014
1,2,15-24,1524
2,3,25-34,2534
3,4,35-44,3544
4,5,45-54,4554
5,6,55-64,5564
6,7,65+,65
7,8,unknown,u


#### Gender

Next, let's create the information table for the genders. There are two unique values, `'m'` and `'f'`. The information table is created by grouping the long-format tidy data set based on `sex` and applying the `.size()` method. The result is assigned to the `gender_info` object.

In [14]:
gender_info = tidy_tb.groupby(['sex']).size().reset_index(name='num_rows')

The `gender_info` table is displayed below. The `.size()` method lets us see the number of rows for each unique value.

In [15]:
gender_info

,sex,num_rows
0,f,46152
1,m,46152


However, those number of rows are specific to the particular data set we are working with. Presumably we would want to continue tracking and recording the number of TB cases over time. Thus, the number of rows is specific to a particular data set and not about the values of interest. Therefore, we will drop the number of rows.

In [16]:
gender_info.drop(columns=['num_rows'], inplace=True)

Let's now add in the unique integer identifier for the genders, just as we had done with the ages.

In [17]:
gender_info['gender_id'] = gender_info.index + 1

Let's reorder the columns so the unique ID is the first column.

In [18]:
gender_info = gender_info.loc[:, ['gender_id', 'sex']].copy()

The gender information table is displayed below.

In [19]:
gender_info

,gender_id,sex
0,1,f
1,2,m


A separate table for a variable with few unique values allows us to store other attributes if we would like. For example, we could use more descriptive names for the genders rather than just `'f'` and `'m'`. You were not required to do this, but the cell below shows how to create the more descriptive name for the genders.

In [20]:
gender_info['gender_name'] = ['female', 'male']

In [21]:
gender_info

,gender_id,sex,gender_name
0,1,f,female
1,2,m,male


#### Year

Let's now go through the same exercise for the `year` variable. We will start the same way by grouping the tidy data set based on the `year` and counting the number of rows for each unique value.

In [22]:
year_info = tidy_tb.groupby(['year']).size().reset_index(name='num_rows')

The first few and last few rows are shown in the cells below. We see that not all years have the same number of rows.

In [23]:
year_info.head()

,year,num_rows
0,1980,3056
1,1981,3104
2,1982,3104
3,1983,3136
4,1984,3088


In [24]:
year_info.tail()

,year,num_rows
24,2004,3312
25,2005,3296
26,2006,3392
27,2007,3392
28,2008,3392


Let's remove the `num_rows` column and then add in the unique ID for each row and then reorganize the column order so the ID is first.

In [25]:
year_info.drop(columns=['num_rows'], axis=1, inplace=True)

In [26]:
year_info['year_id'] = year_info.index + 1

In [27]:
year_info = year_info.loc[:, ['year_id', 'year']].copy()

The first few rows are given below.

In [28]:
year_info.head()

,year_id,year
0,1,1980
1,2,1981
2,3,1982
3,4,1983
4,5,1984


We may not necessarily need to create the `year_id` variable, since the year could be viewed as the integer that unique defines the row. However, we will include in this assignment to give extra practice working with unique IDs. 

We can also consider if we even need a separate "information" table like this for the year. Ultimately, that would come down to the application of interest, but consider if we would like to store information about attributes or features associated with each year. A `year_info` table will allow us to record major global events, such a pandemic or financial crises, and join such events with our other tables.

#### Country

Let's now create a table for the unique countries in the `tidy_tb` data set.

In [29]:
country_info = tidy_tb.groupby(['country']).size().reset_index(name='num_rows')

Displaying the table shows the head and tail by default because the number of rows is in the several hundreds. We see that not every country has the same number of rows in `tidy_tb`. We also see that the countries are identified by two letter codes rather than the full country name.

In [30]:
country_info

,country,num_rows
0,AD,304
1,AE,416
2,AF,384
3,AG,432
4,AI,352
...,...,...
207,YE,336
208,YU,400
209,ZA,464
210,ZM,432


You were not required to do this, but if we would like to confirm that the countries have just 2 letters in their codes, we would want to count the number of characters in each string. We can do that in Pandas with the `.str.len()` method. The cell below shows how to apply the method to the `country` column in `country_info`.

In [31]:
country_info.country.str.len()

0      2
1      2
2      2
3      2
4      2
      ..
207    2
208    2
209    2
210    2
211    2
Name: country, Length: 212, dtype: int64

And then we can apply `.value_counts()` to check the number of unique values for the string length. As shown below, all `country` strings have just 2 characters.

In [32]:
country_info.country.str.len().value_counts()

2    212
Name: country, dtype: int64

Let's drop the `num_rows` column and again include an integer as a unique identifer for each country.

In [33]:
country_info.drop(columns=['num_rows'], axis=1, inplace=True)

In [34]:
country_info['country_id'] = country_info.index + 1

In [35]:
country_info = country_info.loc[:, ['country_id', 'country']].copy()

In [36]:
country_info

,country_id,country
0,1,AD
1,2,AE
2,3,AF
3,4,AG
4,5,AI
...,...,...
207,208,YE
208,209,YU
209,210,ZA
210,211,ZM


### Redefine the tidy data

Now that we have created the separate information tables, let's bring in the unique row identifier integers into the tidy long-format data set. This will allow us to replace the variable descriptive names and abbreviations with the unique integer IDs. We will do this through a series of JOINS. In each of the joins the larger tidy long-format data set will be treated as the "left" data set and the smaller information tables will be the "right" data sets. 

Let's start with the `gender_info` table. The `sex` column exists in both `tidy_tb` and `gender_info`. Therefore, we can join the two data sets **on** `sex`. Since both unique values of `sex` are in the left and right data sets, it will not matter whether we use a left, right, or inner join for this application. I typically like to use the left-join and so the `how` argument in `pd.merge()` will be set to `'left'`.

The cell below executes the join but does not assign the result to an object. The resulting table is displayed to the screen to show that the `gender_id` and `gender_name` columns have been brought into the tidy long-format data set.

In [37]:
pd.merge(tidy_tb, gender_info, on='sex', how='left')

,country,year,sex,age,cases,gender_id,gender_name
0,AD,1989,f,0-14,NaN,1,female
1,AD,1990,f,0-14,NaN,1,female
2,AD,1991,f,0-14,NaN,1,female
3,AD,1992,f,0-14,NaN,1,female
4,AD,1993,f,0-14,NaN,1,female
...,...,...,...,...,...,...,...
92299,NaN,2004,m,unknown,NaN,2,male
92300,NaN,2005,m,unknown,NaN,2,male
92301,NaN,2006,m,unknown,NaN,2,male
92302,NaN,2007,m,unknown,NaN,2,male


As a check, the cell below displays the number of rows and columns in the `tidy_tb` data set. Notice that we have the same number of rows in the joined data set, but two additional columns have been added by joining `gender_info` to `tidy_tb`.

In [38]:
tidy_tb.shape

(92304, 5)

Next, let's chain our join with a join to the `age_info` table. We will use the `.merge()` method associated with the result of the `pd.merge()` function. The joining procedure results in a Pandas DataFrame and thus we have access to the same methods and attributes as any Pandas DataFrame. 

However, before performing the join let's double check the names of the variables in the data sets.

In [39]:
print( age_info.columns )

print( tidy_tb.columns )

Index(['age_id', 'age_group', 'original_tb_word'], dtype='object')
Index(['country', 'year', 'sex', 'age', 'cases'], dtype='object')


The `age` column exists in the `tidy_tb` data set, but does not exist in the `age_info` data set. In `age_info`, the age groups are named `age_group`. We therefore cannot simply join the two data sets by specifying the `on` argument in the `.merge()` method to be `age` or `age_group`. If we would set `on='age'` we would get an error, as shown by the cell below.

In [40]:
pd.merge(tidy_tb, gender_info, on='sex', how='left').\
merge(age_info, on='age', how='left')

KeyError: 'age'

Notice that the error message is `KeyError: 'age'`. That error message is saying there's something wrong with the keys that joined the two data sets on. We can overcome this issue a few ways. One approach is to simply rename the `age_group` column in `age_info` to be `age`, or to rename the `age` column in `tidy_tb` as `age_group`. This approach is perfectly valid and if you had such an issue in your solution and you renamed your columns to match you would receive full credit.

Alternatively, we can specify separate variables as the joining keys in the left and right data sets. We did not discuss this approach in lecture, so you did not need to use this approach in the assignment. However, it can be useful to see and so it is demonstrated here. Instead of specifying a single key with the `on` argument, we can specify the name of the key in the left data set with `left_on` and the name of the key in the right data set with `right_on`. In our example, the "left key" comes from the merge of the `tidy_tb` and `gender_info` and thus is `age` and the "right key" is `age_group`. The cell below shows how to perform the join using the two separate key names.

In [41]:
pd.merge(tidy_tb, gender_info, on='sex', how='left').\
merge(age_info, left_on='age', right_on='age_group', how='left')

,country,year,sex,age,cases,gender_id,gender_name,age_id,age_group,original_tb_word
0,AD,1989,f,0-14,NaN,1,female,1,0-14,014
1,AD,1990,f,0-14,NaN,1,female,1,0-14,014
2,AD,1991,f,0-14,NaN,1,female,1,0-14,014
3,AD,1992,f,0-14,NaN,1,female,1,0-14,014
4,AD,1993,f,0-14,NaN,1,female,1,0-14,014
...,...,...,...,...,...,...,...,...,...,...
92299,NaN,2004,m,unknown,NaN,2,male,8,unknown,u
92300,NaN,2005,m,unknown,NaN,2,male,8,unknown,u
92301,NaN,2006,m,unknown,NaN,2,male,8,unknown,u
92302,NaN,2007,m,unknown,NaN,2,male,8,unknown,u


Notice that the resulting joined data set includes the left and right keys! We therefore have both `age` and `age_group`. The two columns are redundant, they store the exact same information in the exact same order. We do not need both. However, we will keep all columns at the moment.

Next, let's bring in the year information table. We will use a left join again with the two DataFrames joined **on** `year`. The cell below continues to "build up" our chained join and displays the results to screen.

In [42]:
pd.merge(tidy_tb, gender_info, on='sex', how='left').\
merge(age_info, left_on='age', right_on='age_group', how='left').\
merge(year_info, on='year', how='left')

,country,year,sex,age,cases,gender_id,gender_name,age_id,age_group,original_tb_word,year_id
0,AD,1989,f,0-14,NaN,1,female,1,0-14,014,10
1,AD,1990,f,0-14,NaN,1,female,1,0-14,014,11
2,AD,1991,f,0-14,NaN,1,female,1,0-14,014,12
3,AD,1992,f,0-14,NaN,1,female,1,0-14,014,13
4,AD,1993,f,0-14,NaN,1,female,1,0-14,014,14
...,...,...,...,...,...,...,...,...,...,...,...
92299,NaN,2004,m,unknown,NaN,2,male,8,unknown,u,25
92300,NaN,2005,m,unknown,NaN,2,male,8,unknown,u,26
92301,NaN,2006,m,unknown,NaN,2,male,8,unknown,u,27
92302,NaN,2007,m,unknown,NaN,2,male,8,unknown,u,28


Lastly, let's merge in the `country_info` table. The data sets are joined **on** `country` with the final merged data set shown below.

In [43]:
pd.merge(tidy_tb, gender_info, on='sex', how='left').\
merge(age_info, left_on='age', right_on='age_group', how='left').\
merge(year_info, on='year', how='left').\
merge(country_info, on='country', how='left')

,country,year,sex,age,cases,gender_id,gender_name,age_id,age_group,original_tb_word,year_id,country_id
0,AD,1989,f,0-14,NaN,1,female,1,0-14,014,10,1.0
1,AD,1990,f,0-14,NaN,1,female,1,0-14,014,11,1.0
2,AD,1991,f,0-14,NaN,1,female,1,0-14,014,12,1.0
3,AD,1992,f,0-14,NaN,1,female,1,0-14,014,13,1.0
4,AD,1993,f,0-14,NaN,1,female,1,0-14,014,14,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
92299,NaN,2004,m,unknown,NaN,2,male,8,unknown,u,25,NaN
92300,NaN,2005,m,unknown,NaN,2,male,8,unknown,u,26,NaN
92301,NaN,2006,m,unknown,NaN,2,male,8,unknown,u,27,NaN
92302,NaN,2007,m,unknown,NaN,2,male,8,unknown,u,28,NaN


We have now merged in the unique integer IDs for each of the separate tables! The final step requires selecting just the ID columns and the `cases` column. The result is assigned to the `cases_in_country` object.

In [44]:
cases_in_country = pd.merge(tidy_tb, gender_info, on='sex', how='left').\
merge(age_info, left_on='age', right_on='age_group', how='left').\
merge(year_info, on='year', how='left').\
merge(country_info, on='country', how='left').\
loc[:, ['country_id', 'year_id', 'gender_id', 'age_id', 'cases']]

The "redefined" tidy data set is displayed below.

In [45]:
cases_in_country

,country_id,year_id,gender_id,age_id,cases
0,1.0,10,1,1,NaN
1,1.0,11,1,1,NaN
2,1.0,12,1,1,NaN
3,1.0,13,1,1,NaN
4,1.0,14,1,1,NaN
...,...,...,...,...,...
92299,NaN,25,2,8,NaN
92300,NaN,26,2,8,NaN
92301,NaN,27,2,8,NaN
92302,NaN,28,2,8,NaN


The above display shows that the `country_id` looks like a float (a decimal). Let's check the data types to be sure and compare to the data types in the `country_info` object. 

In [46]:
print('-- cases_in_country data types --')
print( cases_in_country.dtypes )

print(' ')
print('-- contry_info data types -- ')
print( country_info.dtypes )

-- cases_in_country data types --
country_id    float64
year_id         int64
gender_id       int64
age_id          int64
cases         float64
dtype: object
 
-- contry_info data types -- 
country_id     int64
country       object
dtype: object


Indeed the data type of `country_id` changed after the merge! This is due to the way the missing value, `NaN`, is handled within Pandas DataFrames.

The last thing we will do is to include a unique identifier for each row in `cases_in_country`.

In [47]:
cases_in_country['id'] = cases_in_country.index + 1

We now have all of the "decomposed" or constituint data sets. Let's save those data sets to CSV files to enable importing into MySQL. We will modify the default behavior of `.to_csv()` so that way missing values are written out as NULL instead of NaN.

In [48]:
cases_in_country.to_csv('cases_in_country_table.csv', index=False, na_rep='NULL')

country_info.to_csv('country_info_table.csv', index=False, na_rep='NULL')

year_info.to_csv('year_info_table.csv', index=False, na_rep='NULL')

gender_info.to_csv('gender_info_table.csv', index=False, na_rep='NULL')

age_info.to_csv('age_info_table.csv', index=False, na_rep='NULL')

As a final check, let's demonstrate recovering the original tidy data set by joining the constituint tables back together. Notice that since the "redefined" long-format data set, `cases_in_country` uses the ID's all of the keys to join the tables are `*_id` variables.

In [49]:
pd.merge(cases_in_country, age_info, on='age_id', how='left').\
merge(gender_info, on='gender_id', how='left').\
merge(year_info, on='year_id', how='left').\
merge(country_info, on='country_id', how='left')

,country_id,year_id,gender_id,age_id,cases,id,age_group,original_tb_word,sex,gender_name,year,country
0,1.0,10,1,1,NaN,1,0-14,014,f,female,1989,AD
1,1.0,11,1,1,NaN,2,0-14,014,f,female,1990,AD
2,1.0,12,1,1,NaN,3,0-14,014,f,female,1991,AD
3,1.0,13,1,1,NaN,4,0-14,014,f,female,1992,AD
4,1.0,14,1,1,NaN,5,0-14,014,f,female,1993,AD
...,...,...,...,...,...,...,...,...,...,...,...,...
92299,NaN,25,2,8,NaN,92300,unknown,u,m,male,2004,NaN
92300,NaN,26,2,8,NaN,92301,unknown,u,m,male,2005,NaN
92301,NaN,27,2,8,NaN,92302,unknown,u,m,male,2006,NaN
92302,NaN,28,2,8,NaN,92303,unknown,u,m,male,2007,NaN


The exact style of the tidy long-format `tidy_tb` DataFrame is created by keeping just a subset of the columns in the re-joined object.

In [50]:
pd.merge(cases_in_country, age_info, on='age_id', how='left').\
merge(gender_info, on='gender_id', how='left').\
merge(year_info, on='year_id', how='left').\
merge(country_info, on='country_id', how='left').\
loc[:, ['country', 'year', 'sex', 'age_group', 'cases']]

,country,year,sex,age_group,cases
0,AD,1989,f,0-14,NaN
1,AD,1990,f,0-14,NaN
2,AD,1991,f,0-14,NaN
3,AD,1992,f,0-14,NaN
4,AD,1993,f,0-14,NaN
...,...,...,...,...,...
92299,NaN,2004,m,unknown,NaN
92300,NaN,2005,m,unknown,NaN
92301,NaN,2006,m,unknown,NaN
92302,NaN,2007,m,unknown,NaN


## Application 2 - Stock data

### Tidy the data

The homework 02 solutions walk through how to tidy the four separate wide-format files into a single tidy long-format data set. We will only repeat the minimum set of operations to recreate the long-format data set here. Please see the homework 02 solutions for detailed discussions about the steps.

In [51]:
### read the data sets
df_open = pd.read_csv("../hw02/daily_open_prices.csv")

df_close = pd.read_csv('../hw02/daily_close_prices.csv')

df_low = pd.read_csv('../hw02/daily_low_prices.csv')

df_high = pd.read_csv('../hw02/daily_high_prices.csv')

### reshape each to long-format

lf_open = df_open.melt( id_vars=['date'], value_vars=df_open.columns.tolist()[1:])

lf_close = df_close.melt( id_vars=['date'], value_vars=df_close.columns.tolist()[1:])

lf_low = df_low.melt( id_vars=['date'], value_vars=df_low.columns.tolist()[1:])

lf_high = df_high.melt( id_vars=['date'], value_vars=df_high.columns.tolist()[1:])

### concatenate all long-format data sets together
lf_stocks = pd.concat([lf_open, lf_close, lf_low, lf_high])

### parse the variable name
lf_stocks[['symbol', 'price_at']] = lf_stocks.variable.str.split( '_', expand=True )

lf_stocks.drop(['variable'], axis=1, inplace=True)

lf_stocks['date_dt'] = pd.to_datetime( lf_stocks.date )

lf_stocks.drop(['date'], axis=1, inplace=True)

The tidy long-format data set in homework 02 was completed by bringing in the informatio contained in the `stock_info` data set. We also extracted the year, month, day, and weekday from the datetime column, `date_dt`. However, we will stop at the moment because we are "decomposing" or "breaking up" the long-format data set into smaller constitutient tables. Our long-format data set is shown below.

In [52]:
lf_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 0 to 599
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   value     2400 non-null   float64       
 1   symbol    2400 non-null   object        
 2   price_at  2400 non-null   object        
 3   date_dt   2400 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 93.8+ KB


In [53]:
lf_stocks

,value,symbol,price_at,date_dt
0,132.759995,AAPL,open,2020-09-01
1,137.589996,AAPL,open,2020-09-02
2,126.910004,AAPL,open,2020-09-03
3,120.070000,AAPL,open,2020-09-04
4,113.949997,AAPL,open,2020-09-08
...,...,...,...,...
595,57.580002,VZ,high,2021-01-19
596,57.389999,VZ,high,2021-01-20
597,57.509998,VZ,high,2021-01-21
598,57.689999,VZ,high,2021-01-22


### Decompose the tidy data

We will now work through breaking up the long-format data set into "smaller piecs". Those small pieces will contain *attributes* associated with unique *instances* or records of *entities*. We will start with the `stock_info` data set which stores information associated with each company.

#### Stock information

The stock data set from homework 02 includes the `stock_info` CSV file. This file already provides the unique information about each stock. Let's read that CSV file into a Pandas DataFrame first.

In [54]:
stock_info = pd.read_csv( '../hw02/stock_info.csv' )

The `.info()` method is displayed below to give the variable names and data types for each column.

In [55]:
stock_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   symbol    6 non-null      object
 1   company   6 non-null      object
 2   sector    6 non-null      object
 3   exchange  6 non-null      object
dtypes: object(4)
memory usage: 320.0+ bytes


Since the `stock_info` data set is small, let's just look at the entire DataFrame. We see that one row is one company.  

In [56]:
stock_info

,symbol,company,sector,exchange
0,AAPL,Apple Inc.,Information Technology,NASDAQ
1,VZ,Verizon Communications Inc.,Communication Services,NYSE
2,KO,Coca-Cola Company,Consumer Staples,NYSE
3,AMZN,Amazon.com Inc.,Consumer Discretionary,NASDAQ
4,TSLA,Tesla Inc,Consumer Discretionary,NASDAQ
5,GME,GameStop Corp.,Consumer Discretionary,NYSE


As we can see each company is associated with a `sector` and an `exchange`. We will break up the `stock_info` table into three distinct tables to reflect these different pieces of information. We will have one table for the company name and symbol, one table for the sector, and another table for the exchange. 

The `company_info` table is created below by grouping by `symbol` and `company`. The `.size()` method is used to count the number of rows associated with each unique combination of the two columns. The `.drop()` method is applied to drop the number of rows column since we are just interested in the company attributes.

In [57]:
company_info = stock_info.groupby(['symbol', 'company']).size().\
reset_index(name='num_rows').\
drop(columns=['num_rows'], axis=1)

In [58]:
company_info

,symbol,company
0,AAPL,Apple Inc.
1,AMZN,Amazon.com Inc.
2,GME,GameStop Corp.
3,KO,Coca-Cola Company
4,TSLA,Tesla Inc
5,VZ,Verizon Communications Inc.


An integer is included to represent the unique company ID in the `company_info` table.

In [59]:
company_info['company_id'] = company_info.index + 1

In [60]:
company_info

,symbol,company,company_id
0,AAPL,Apple Inc.,1
1,AMZN,Amazon.com Inc.,2
2,GME,GameStop Corp.,3
3,KO,Coca-Cola Company,4
4,TSLA,Tesla Inc,5
5,VZ,Verizon Communications Inc.,6


The sector and exchange information tables are created in similar fashion.

In [61]:
sector_info = stock_info.groupby(['sector']).size().\
reset_index(name='num_rows').\
drop(columns=['num_rows'], axis=1)

In [62]:
sector_info['sector_id'] = sector_info.index + 1

In [63]:
sector_info

,sector,sector_id
0,Communication Services,1
1,Consumer Discretionary,2
2,Consumer Staples,3
3,Information Technology,4


In [64]:
exchange_info = stock_info.groupby(['exchange']).size().\
reset_index(name='num_rows').\
drop(columns=['num_rows'], axis=1)

In [65]:
exchange_info['exchange_id'] = exchange_info.index + 1

In [66]:
exchange_info

,exchange,exchange_id
0,NASDAQ,1
1,NYSE,2


A benefit of having separate tables for *entities* like the exchanges is that we can store the primary address, time zone, and other *attributes* associated with the exchange. Those pieces of information are specific to the exchange and thus can be stored separately from the information related to the companies.

#### Price types

Let's now decompose the the types of prices stored in the long-format data set. We know that there are 4 prices in our current application, the open, close, low, and high prices. The long-format in theory could store the price at every second of the trading if we wanted! In that case we could view the `price_at` column as analogous to a timestamp. However, in our current application we are storing summary statistics about the price on a given day. Let's name our table `price_type_info` and create it by grouping the `lf_stocks` object by the `price_at` column.

In [67]:
price_type_info = lf_stocks.groupby(['price_at']).size().reset_index(name='num_rows')

Displaying the `price_type_info` table below reveals that we have 600 rows for each type of price. As explained in the homework 02 solutions, we have 6 companies each with 100 trading days worth of data. That is why there are 600 rows for each type of price in the long-format data set.

In [68]:
price_type_info

,price_at,num_rows
0,close,600
1,high,600
2,low,600
3,open,600


Let's drop the `num_rows` column and then include an integer ID for the type of price.

In [69]:
price_type_info.drop(columns=['num_rows'], axis=1, inplace=True)

In [70]:
price_type_info['price_type_id'] = price_type_info.index + 1

In [71]:
price_type_info

,price_at,price_type_id
0,close,1
1,high,2
2,low,3
3,open,4


#### Trading dates

We could also create a separate information table for the trading days for this application. However, we will not do that for the stocks data. The trading day will be viewed as an *attribute* of the long-format data itself. We are therefore recording the value of a type of price for each stock on a day. The day and value are recorded together. Creating a separate information table for the date is allowable and will still receive full credit. The solution presented here is just showing one possible way to structure the *database*.

### Redefine the tidy data

#### Stock info

Now that we have created our separate information tables, let's redefine our tiday data in terms of the unique IDs. We will start with the `stock_info` table. We wish to store the information present in the `stock_info` table in the three created tables `company_info`, `sector_info`, and `exchange_info`. The "long-format" stock information table will then store just the *linking* between the three tables. We will name this table the `company_sector_exchange` table since those are three pieces of information that is stores.

The `company_sector_exchange` table is created by a series of joins starting with `stock_info` as the left table in the join. The cell below performs the necessary joining steps chained together to bring in the ID variables into the `stock_info` table. The result is displayed to the screen to show the structure of the data set.

In [72]:
pd.merge( stock_info, company_info, on=['symbol', 'company'], how='left').\
merge( sector_info, on='sector', how='left').\
merge( exchange_info, on='exchange', how='left')

,symbol,company,sector,exchange,company_id,sector_id,exchange_id
0,AAPL,Apple Inc.,Information Technology,NASDAQ,1,4,1
1,VZ,Verizon Communications Inc.,Communication Services,NYSE,6,1,2
2,KO,Coca-Cola Company,Consumer Staples,NYSE,4,3,2
3,AMZN,Amazon.com Inc.,Consumer Discretionary,NASDAQ,2,2,1
4,TSLA,Tesla Inc,Consumer Discretionary,NASDAQ,5,2,1
5,GME,GameStop Corp.,Consumer Discretionary,NYSE,3,2,2


Let's now assign the result of the joins to the `company_sector_exchange` table, after dropping all but the ID columns.

In [73]:
company_sector_exchange = pd.merge( stock_info, company_info, on=['symbol', 'company'], how='left').\
merge( sector_info, on='sector', how='left').\
merge( exchange_info, on='exchange', how='left').\
loc[:, ['company_id', 'sector_id', 'exchange_id']]

Lastly, let's include an integer as a unique ID within the *linking* table.

In [74]:
company_sector_exchange['link_id'] = company_sector_exchange.index + 1

In [75]:
company_sector_exchange

,company_id,sector_id,exchange_id,link_id
0,1,4,1,1
1,6,1,2,2
2,4,3,2,3
3,2,2,1,4
4,5,2,1,5
5,3,2,2,6


#### Long-format prices

Let's now bring in the unique IDs for the companies and the types of prices into the long-format stock data set. Again we will use a series of joins to accomplish this. Because have separate tables for the companies, sectors, and exchanges, we only need to join the `company_info` to `lf_stocks`. The two DataFrames are joined **on** the `symbol` column, as shown below.

In [76]:
pd.merge( lf_stocks, company_info, on='symbol', how='left')

,value,symbol,price_at,date_dt,company,company_id
0,132.759995,AAPL,open,2020-09-01,Apple Inc.,1
1,137.589996,AAPL,open,2020-09-02,Apple Inc.,1
2,126.910004,AAPL,open,2020-09-03,Apple Inc.,1
3,120.070000,AAPL,open,2020-09-04,Apple Inc.,1
4,113.949997,AAPL,open,2020-09-08,Apple Inc.,1
...,...,...,...,...,...,...
2395,57.580002,VZ,high,2021-01-19,Verizon Communications Inc.,6
2396,57.389999,VZ,high,2021-01-20,Verizon Communications Inc.,6
2397,57.509998,VZ,high,2021-01-21,Verizon Communications Inc.,6
2398,57.689999,VZ,high,2021-01-22,Verizon Communications Inc.,6


The type of prices ID is brought into the `lf_stocks` DataFrame by joining `price_type_info` **on** `price_at`.

In [77]:
pd.merge( lf_stocks, company_info, on='symbol', how='left').\
merge(price_type_info, on='price_at', how='left')

,value,symbol,price_at,date_dt,company,company_id,price_type_id
0,132.759995,AAPL,open,2020-09-01,Apple Inc.,1,4
1,137.589996,AAPL,open,2020-09-02,Apple Inc.,1,4
2,126.910004,AAPL,open,2020-09-03,Apple Inc.,1,4
3,120.070000,AAPL,open,2020-09-04,Apple Inc.,1,4
4,113.949997,AAPL,open,2020-09-08,Apple Inc.,1,4
...,...,...,...,...,...,...,...
2395,57.580002,VZ,high,2021-01-19,Verizon Communications Inc.,6,2
2396,57.389999,VZ,high,2021-01-20,Verizon Communications Inc.,6,2
2397,57.509998,VZ,high,2021-01-21,Verizon Communications Inc.,6,2
2398,57.689999,VZ,high,2021-01-22,Verizon Communications Inc.,6,2


The redefined long-format data set is then created by selecting the ID columns, the date column, and the value column. The result is assigned to the `stock_value_per_day` table. 

In [78]:
stock_value_per_day = pd.merge( lf_stocks, company_info, on='symbol', how='left').\
merge(price_type_info, on='price_at', how='left').\
loc[:, ['company_id', 'price_type_id', 'date_dt', 'value']]

Lastly, let's define an integer as the unique ID for each row in `stock_value_per_day`.

In [79]:
stock_value_per_day['id'] = stock_value_per_day.index + 1

The `.info()` method and a display of the redefined long-format data set are given below.

In [80]:
stock_value_per_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 0 to 2399
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   company_id     2400 non-null   int64         
 1   price_type_id  2400 non-null   int64         
 2   date_dt        2400 non-null   datetime64[ns]
 3   value          2400 non-null   float64       
 4   id             2400 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 112.5 KB


In [81]:
stock_value_per_day

,company_id,price_type_id,date_dt,value,id
0,1,4,2020-09-01,132.759995,1
1,1,4,2020-09-02,137.589996,2
2,1,4,2020-09-03,126.910004,3
3,1,4,2020-09-04,120.070000,4
4,1,4,2020-09-08,113.949997,5
...,...,...,...,...,...
2395,6,2,2021-01-19,57.580002,2396
2396,6,2,2021-01-20,57.389999,2397
2397,6,2,2021-01-21,57.509998,2398
2398,6,2,2021-01-22,57.689999,2399


We now have all of the necessary data sets for our database. Let's save each of the constituent tables to CSV files to enable importing into MySQL.

In [82]:
exchange_info.to_csv('exchange_info_table.csv', index=False, na_rep='NULL')

sector_info.to_csv('sector_info_table.csv', index=False, na_rep='NULL')

company_info.to_csv('company_info_table.csv', index=False, na_rep='NULL')

company_sector_exchange.to_csv('company_sector_exchange_table.csv', index=False, na_rep='NULL')

price_type_info.to_csv('price_type_info_table.csv', index=False, na_rep='NULL')

stock_value_per_day.to_csv('stock_value_per_day_table.csv', index=False, na_rep='NULL')

The original tidy data set can be recreated by joining the necessary information tables together. The joining keys are now the unique ID variables which serve to link the tables together. The joins are performed in the cell below, starting with the `stock_value_per_day` as the left table, since it is the largest of the tables. This follows the convention of joining smaller tables, as the "right" table, to larger tables on the "left.

In [83]:
pd.merge( stock_value_per_day, price_type_info, on='price_type_id', how='left').\
merge(company_info, on='company_id', how='left')

,company_id,price_type_id,date_dt,value,id,price_at,symbol,company
0,1,4,2020-09-01,132.759995,1,open,AAPL,Apple Inc.
1,1,4,2020-09-02,137.589996,2,open,AAPL,Apple Inc.
2,1,4,2020-09-03,126.910004,3,open,AAPL,Apple Inc.
3,1,4,2020-09-04,120.070000,4,open,AAPL,Apple Inc.
4,1,4,2020-09-08,113.949997,5,open,AAPL,Apple Inc.
...,...,...,...,...,...,...,...,...
2395,6,2,2021-01-19,57.580002,2396,high,VZ,Verizon Communications Inc.
2396,6,2,2021-01-20,57.389999,2397,high,VZ,Verizon Communications Inc.
2397,6,2,2021-01-21,57.509998,2398,high,VZ,Verizon Communications Inc.
2398,6,2,2021-01-22,57.689999,2399,high,VZ,Verizon Communications Inc.


We can then bring in the sector and exchange information by including the `company_sector_exchange` table that relates each company to a sector and exchange. Notice that the join is performed **on** `company_id` and the result now includes the `sector_id` and `exchange_id` columns.

In [84]:
pd.merge( stock_value_per_day, price_type_info, on='price_type_id', how='left').\
merge(company_info, on='company_id', how='left').\
merge(company_sector_exchange, on='company_id', how='left')

,company_id,price_type_id,date_dt,value,id,price_at,symbol,company,sector_id,exchange_id,link_id
0,1,4,2020-09-01,132.759995,1,open,AAPL,Apple Inc.,4,1,1
1,1,4,2020-09-02,137.589996,2,open,AAPL,Apple Inc.,4,1,1
2,1,4,2020-09-03,126.910004,3,open,AAPL,Apple Inc.,4,1,1
3,1,4,2020-09-04,120.070000,4,open,AAPL,Apple Inc.,4,1,1
4,1,4,2020-09-08,113.949997,5,open,AAPL,Apple Inc.,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2395,6,2,2021-01-19,57.580002,2396,high,VZ,Verizon Communications Inc.,1,2,2
2396,6,2,2021-01-20,57.389999,2397,high,VZ,Verizon Communications Inc.,1,2,2
2397,6,2,2021-01-21,57.509998,2398,high,VZ,Verizon Communications Inc.,1,2,2
2398,6,2,2021-01-22,57.689999,2399,high,VZ,Verizon Communications Inc.,1,2,2


The sector and exchange information are brought in by performing two more joins to those necessary tables.

In [85]:
pd.merge( stock_value_per_day, price_type_info, on='price_type_id', how='left').\
merge(company_info, on='company_id', how='left').\
merge(company_sector_exchange, on='company_id', how='left').\
merge(sector_info, on='sector_id', how='left').\
merge(exchange_info, on='exchange_id', how='left')

,company_id,price_type_id,date_dt,value,id,price_at,symbol,company,sector_id,exchange_id,link_id,sector,exchange
0,1,4,2020-09-01,132.759995,1,open,AAPL,Apple Inc.,4,1,1,Information Technology,NASDAQ
1,1,4,2020-09-02,137.589996,2,open,AAPL,Apple Inc.,4,1,1,Information Technology,NASDAQ
2,1,4,2020-09-03,126.910004,3,open,AAPL,Apple Inc.,4,1,1,Information Technology,NASDAQ
3,1,4,2020-09-04,120.070000,4,open,AAPL,Apple Inc.,4,1,1,Information Technology,NASDAQ
4,1,4,2020-09-08,113.949997,5,open,AAPL,Apple Inc.,4,1,1,Information Technology,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,6,2,2021-01-19,57.580002,2396,high,VZ,Verizon Communications Inc.,1,2,2,Communication Services,NYSE
2396,6,2,2021-01-20,57.389999,2397,high,VZ,Verizon Communications Inc.,1,2,2,Communication Services,NYSE
2397,6,2,2021-01-21,57.509998,2398,high,VZ,Verizon Communications Inc.,1,2,2,Communication Services,NYSE
2398,6,2,2021-01-22,57.689999,2399,high,VZ,Verizon Communications Inc.,1,2,2,Communication Services,NYSE


Finally, the tidy data set from homework 02 is recovered by dropping the ID columns.

In [86]:
pd.merge( stock_value_per_day, price_type_info, on='price_type_id', how='left').\
merge(company_info, on='company_id', how='left').\
merge(company_sector_exchange, on='company_id', how='left').\
merge(sector_info, on='sector_id', how='left').\
merge(exchange_info, on='exchange_id', how='left').\
loc[:, ['exchange', 'sector', 'company', 'symbol', 'date_dt', 'price_at', 'value']]

,exchange,sector,company,symbol,date_dt,price_at,value
0,NASDAQ,Information Technology,Apple Inc.,AAPL,2020-09-01,open,132.759995
1,NASDAQ,Information Technology,Apple Inc.,AAPL,2020-09-02,open,137.589996
2,NASDAQ,Information Technology,Apple Inc.,AAPL,2020-09-03,open,126.910004
3,NASDAQ,Information Technology,Apple Inc.,AAPL,2020-09-04,open,120.070000
4,NASDAQ,Information Technology,Apple Inc.,AAPL,2020-09-08,open,113.949997
...,...,...,...,...,...,...,...
2395,NYSE,Communication Services,Verizon Communications Inc.,VZ,2021-01-19,high,57.580002
2396,NYSE,Communication Services,Verizon Communications Inc.,VZ,2021-01-20,high,57.389999
2397,NYSE,Communication Services,Verizon Communications Inc.,VZ,2021-01-21,high,57.509998
2398,NYSE,Communication Services,Verizon Communications Inc.,VZ,2021-01-22,high,57.689999


All we would need to do is extract the year, month, day, and week day information from the date time column to have the exact same data set from homework 02. 